In [ ]:
!pip install tensorflow matplotlib seaborn scikit-learn

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetV2B0
from tensorflow.keras.layers import (
    Dense, GlobalAveragePooling2D, Dropout,
    BatchNormalization, MultiHeadAttention,
    LayerNormalization, Add
)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
root_path = "/content/drive/MyDrive/AnnotatedUltrasoundLiver_Dataset"

In [6]:
import os
import shutil

root_path = "/content/drive/MyDrive/AnnotatedUltrasoundLiver_Dataset"
clean_dataset_path = "/content/LiverDataset"

os.makedirs(clean_dataset_path, exist_ok=True)

print("Scanning dataset...\n")

for class_name in os.listdir(root_path):
    class_path = os.path.join(root_path, class_name)

    if os.path.isdir(class_path):
        image_folder = os.path.join(class_path, "image")

        if os.path.exists(image_folder):
            print(f"Found class: {class_name}")

            new_class_path = os.path.join(clean_dataset_path, class_name)
            os.makedirs(new_class_path, exist_ok=True)

            for img in os.listdir(image_folder):
                if img.lower().endswith(('.png', '.jpg', '.jpeg')):
                    src = os.path.join(image_folder, img)
                    dst = os.path.join(new_class_path, img)
                    shutil.copy(src, dst)

print("\nDataset cleaned successfully!")

Scanning dataset...

Found class: Benign
Found class: Malignant
Found class: Normal

Dataset cleaned successfully!


In [7]:
for cls in os.listdir("/content/LiverDataset"):
    print(cls, "->", len(os.listdir(os.path.join("/content/LiverDataset", cls))), "images")

Normal -> 100 images
Benign -> 200 images
Malignant -> 435 images


In [8]:
img_size = 224
batch_size = 16

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=25,
    zoom_range=0.2,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = datagen.flow_from_directory(
    clean_dataset_path,
    target_size=(img_size,img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_generator = datagen.flow_from_directory(
    clean_dataset_path,
    target_size=(img_size,img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

Found 588 images belonging to 3 classes.
Found 147 images belonging to 3 classes.


In [9]:
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)

class_weights = dict(enumerate(class_weights))
print(class_weights)

{0: np.float64(1.225), 1: np.float64(0.5632183908045977), 2: np.float64(2.45)}


In [10]:
base_model = EfficientNetV2B0(
    weights='imagenet',
    include_top=False,
    input_shape=(224,224,3)
)

base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)

# Reshape for attention
x = tf.keras.layers.Reshape((1, x.shape[-1]))(x)

attention_output = MultiHeadAttention(
    num_heads=4,
    key_dim=64
)(x, x)

x = Add()([x, attention_output])
x = LayerNormalization()(x)

x = tf.keras.layers.Flatten()(x)
x = Dropout(0.5)(x)

output = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

24274472/24274472 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [11]:
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [12]:
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.3,
    patience=3,
    verbose=1
)

In [14]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    class_weight=class_weights,
    callbacks=[early_stop, reduce_lr]
)

Epoch 1/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 52s 1s/step - accuracy: 0.3400 - loss: 1.8392 - val_accuracy: 0.1361 - val_loss: 2.3036 - learning_rate: 1.0000e-04
Epoch 2/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 49s 1s/step - accuracy: 0.2899 - loss: 1.6708 - val_accuracy: 0.1361 - val_loss: 2.0422 - learning_rate: 1.0000e-04
Epoch 3/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 49s 1s/step - accuracy: 0.3287 - loss: 1.7160 - val_accuracy: 0.1361 - val_loss: 2.1792 - learning_rate: 1.0000e-04
Epoch 4/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 49s 1s/step - accuracy: 0.3177 - loss: 1.6621 - val_accuracy: 0.1361 - val_loss: 1.8188 - learning_rate: 1.0000e-04
Epoch 5/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 49s 1s/step - accuracy: 0.3049 - loss: 1.6448 - val_accuracy: 0.1361 - val_loss: 1.7831 - learning_rate: 1.0000e-04
Epoch 6/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 81s 1s/step - accuracy: 0.4026 - loss: 1.4102 - val_accuracy: 0.1361 - val_loss: 1.2867 - learning_rate: 1.0000e-04
Epoch 7/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 49s 1s/step - accuracy: 0.3133 - loss:

In [17]:
base_model.trainable = True

for layer in base_model.layers[:-30]:
    layer.trainable = False

model.compile(
    optimizer=Adam(1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history_fine = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    class_weight=class_weights,
    callbacks=[early_stop, reduce_lr]
)

Epoch 1/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 80s 2s/step - accuracy: 0.3354 - loss: 1.7310 - val_accuracy: 0.5918 - val_loss: 0.9593 - learning_rate: 1.0000e-05
Epoch 2/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 56s 2s/step - accuracy: 0.3202 - loss: 1.5074 - val_accuracy: 0.5918 - val_loss: 0.9639 - learning_rate: 1.0000e-05
Epoch 3/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 56s 2s/step - accuracy: 0.3008 - loss: 1.5468 - val_accuracy: 0.5918 - val_loss: 0.9367 - learning_rate: 1.0000e-05
Epoch 4/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 81s 1s/step - accuracy: 0.3102 - loss: 1.4623 - val_accuracy: 0.5918 - val_loss: 0.9387 - learning_rate: 1.0000e-05
Epoch 5/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 56s 1s/step - accuracy: 0.3342 - loss: 1.5057 - val_accuracy: 0.5918 - val_loss: 0.9857 - learning_rate: 1.0000e-05
Epoch 6/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3415 - loss: 1.3429
Epoch 6: ReduceLROnPlateau reducing learning rate to 2.9999999242136253e-06.
37/37 ━━━━━━━━━━━━━━━━━━━━ 56s 2s/step - accuracy: 0.3420 - loss: 1.34

In [18]:
val_generator.reset()
pred = model.predict(val_generator)
y_pred = np.argmax(pred, axis=1)

print(classification_report(
    val_generator.classes,
    y_pred,
    target_names=list(train_generator.class_indices.keys())
))

10/10 ━━━━━━━━━━━━━━━━━━━━ 25s 2s/step
              precision    recall  f1-score   support

      Benign       0.00      0.00      0.00        40
   Malignant       0.59      1.00      0.74        87
      Normal       0.00      0.00      0.00        20

    accuracy                           0.59       147
   macro avg       0.20      0.33      0.25       147
weighted avg       0.35      0.59      0.44       147



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
